In [1]:
from clearml import Task, TaskTypes, Dataset, Logger
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib

/tmp/ipykernel_68207/2746575514.py:3: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


#### Load Dataset

In [2]:
dataset = Dataset.get('db392699ef964cbf980c40355e7b21d7')
download_path = dataset.get_local_copy()
print(download_path)

data = pd.read_csv(download_path + '/Claim Denial Updated V7.csv')
data.shape

/home/abhijitbarman/.clearml/cache/storage_manager/datasets/ds_db392699ef964cbf980c40355e7b21d7


(5600, 64)

In [3]:
columns_to_remove = ['Unnamed: 0' ,'CLAIM_TYPE', 'CLAIM_ID' , 'CLAIM_LINE_NUMBER', 'ENCOUNTER_ID', 'PATIENT_ID', 'Name',
'CLAIM_START_DATE', 'CLAIM_END_DATE', 'CLAIM_LINE_START_DATE','CLAIM_LINE_END_DATE',
'PLACE_OF_SERVICE_DESCRIPTION','REVENUE_CENTER_DESCRIPTION','RENDERING_NPI',
'FACILITY_NPI','DISCHARGE_DISPOSITION_DESCRIPTION','DATA_SOURCE','COVERAGE_START_DATE',
'COVERAGE_END_DATE','PAYER', 'PAYER_TYPE', 'BIRTH_DATE','ZIP_CODE','DECEASED_FLAG',
'CONDITION_DATE','CONDITION_TYPE','CODE_TYPE','DESCRIPTION','MONTH','YEAR','ENCOUNTER_START_DATE',
'ENCOUNTER_END_DATE','ADMIT_SOURCE_DESCRIPTION','ADMIT_TYPE_DESCRIPTION',
'PROCEDURE_DATE', 'Year','PHYSICIAN_NPI', 'DENIAL_CATEGORY','APPROVED_CHARGE','CLAIM_STATUS'] 

print(len(columns_to_remove))

40


In [4]:
col_removed_df = data.drop(columns_to_remove, axis=1)

In [5]:
col_removed_df.shape

(5600, 24)

In [ ]:
""" #DENIAL_CATEGORY is Empty for all Approved Claims.So, Set DENIAL_CATEGORY to NOT_APPLICABLE for all 'Approved'  Claims
col_removed_df['DENIAL_CATEGORY'] = col_removed_df['DENIAL_CATEGORY'].fillna('NOT_APPLICABLE')

col_removed_df['DENIAL_CATEGORY'].replace('Prior Authorization ','Prior Authorization', inplace=True)
col_removed_df['DENIAL_CATEGORY'].replace('Prior AUthorization','Prior Authorization', inplace=True)

col_removed_df['DENIAL_CATEGORY'].value_counts() """

In [6]:
#Add a new REASON_CODE='APPROVED' for Approved Claims
col_removed_df['REASON_CODE'].fillna(1.0, inplace=True)

In [9]:
numerical_df = col_removed_df.select_dtypes(include='number')
categorical_df = col_removed_df.select_dtypes(include='object')

cat_col_from_numerical_df = ['PLACE_OF_SERVICE_CODE' , 'REVENUE_CENTER_CODE', 'DISCHARGE_DISPOSITION_CODE', 'MEDICARE_STATUS',
                             'ADMIT_SOURCE_CODE','ADMIT_TYPE_CODE', 'MS_DRG']

categorcal_cols = categorical_df.columns.tolist()
categorcal_cols.extend(cat_col_from_numerical_df)
categorcal_cols, len(categorcal_cols)

(['BILL_TYPE_CODE',
  'HCPCS_CODE',
  'GENDER',
  'RACE',
  'STATE',
  'COUNTY',
  'CODE',
  'PRESENT_ON_ADMIT',
  'DUAL_STATUS',
  'ENCOUNTER_TYPE',
  'PAYERS',
  'PROCEDURE_CODE',
  'PROCEDURE_DESCRIPTION',
  'PLACE_OF_SERVICE_CODE',
  'REVENUE_CENTER_CODE',
  'DISCHARGE_DISPOSITION_CODE',
  'MEDICARE_STATUS',
  'ADMIT_SOURCE_CODE',
  'ADMIT_TYPE_CODE',
  'MS_DRG'],
 20)

In [10]:
numerical_df.columns

Index(['PLACE_OF_SERVICE_CODE', 'REVENUE_CENTER_CODE', 'SERVICE_UNIT_QUANTITY',
       'TOTAL_CHARGES', 'DISCHARGE_DISPOSITION_CODE', 'Age', 'MEDICARE_STATUS',
       'ADMIT_SOURCE_CODE', 'ADMIT_TYPE_CODE', 'MS_DRG', 'REASON_CODE'],
      dtype='object')

In [11]:
categorical_df =  col_removed_df[categorcal_cols]
numerical_cols = list(set(col_removed_df.columns).difference(set(categorical_df.columns)))
numerical_df = col_removed_df[numerical_cols]
numerical_cols

['Age', 'SERVICE_UNIT_QUANTITY', 'TOTAL_CHARGES', 'REASON_CODE']

In [12]:
categorical_df.columns, numerical_cols

(Index(['BILL_TYPE_CODE', 'HCPCS_CODE', 'GENDER', 'RACE', 'STATE', 'COUNTY',
        'CODE', 'PRESENT_ON_ADMIT', 'DUAL_STATUS', 'ENCOUNTER_TYPE', 'PAYERS',
        'PROCEDURE_CODE', 'PROCEDURE_DESCRIPTION', 'PLACE_OF_SERVICE_CODE',
        'REVENUE_CENTER_CODE', 'DISCHARGE_DISPOSITION_CODE', 'MEDICARE_STATUS',
        'ADMIT_SOURCE_CODE', 'ADMIT_TYPE_CODE', 'MS_DRG'],
       dtype='object'),
 ['Age', 'SERVICE_UNIT_QUANTITY', 'TOTAL_CHARGES', 'REASON_CODE'])

In [13]:
merged = pd.concat([numerical_df, categorical_df],axis=1 )
merged.shape

(5600, 24)

In [14]:
#There are duplicate rows. We'll be removing those.
merged_no_dup = merged.drop_duplicates()
merged_no_dup.shape

(5571, 24)

In [15]:
#profile = ProfileReport(merged_no_dup, title="CLAIMS DENIAL ANALYSIS",html={"style":{"full_width":True}})
#profile.to_file(output_file="cliams_denial_copy.html")

task = Task.init('RCM_Claims_Denial_mgmt','EDA_Pandas_Profiling',task_type=TaskTypes.data_processing)

""" Logger.current_logger().report_media(
   "html", "cliams_denial_pandas_profile", iteration=0, local_path="cliams_denial_copy.html"
) """
#task.close()

ClearML Task: overwriting (reusing) task id=e951dae3b9ad414a9b8ff59b8fd6c90f
2023-05-11 14:34:35,821 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2023-05-11 14:34:35,830 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: http://3.111.229.37:8080/projects/37a0acef88f847c1a5af15a811add6d7/experiments/e951dae3b9ad414a9b8ff59b8fd6c90f/output/log


' Logger.current_logger().report_media(\n   "html", "cliams_denial_pandas_profile", iteration=0, local_path="cliams_denial_copy.html"\n) '

ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [16]:
unique_names = {v:'PD_'+str(i) for i, v in enumerate(merged_no_dup['PROCEDURE_DESCRIPTION'].unique().tolist())}
pd.concat([pd.Series(unique_names.keys()), pd.Series(unique_names.values())],axis=1).to_csv('PROCEDURE_DESCRIPTION_mapping.csv')
unique_names

{'Charges do not meet qualifications for emergent/urgent care.': 'PD_0',
 'Processed in Excess of charges.': 'PD_1',
 'Lifetime benefit maximum has been reached for this service/benefit category.': 'PD_2',
 'ESRD network support adjustment.': 'PD_3',
 'The procedure code is inconsistent with the provider type/specialty (taxonomy).': 'PD_4',
 'This care may be covered by another payer per coordination of benefits.': 'PD_5',
 'Expenses incurred prior to coverage.': 'PD_6',
 'Patient has not met the required eligibility requirements.': 'PD_7',
 'Revenue code and Procedure code do not match.': 'PD_8',
 'Charges are covered under a capitation agreement/managed care plan.': 'PD_9',
 'The diagnosis is inconsistent with the procedure.': 'PD_10',
 "These are non-covered services because this is not deemed a 'medical necessity' by the payer.": 'PD_11',
 'Procedure code was invalid on the date of service.': 'PD_12',
 'Lifetime benefit maximum has been reached.': 'PD_13',
 'Benefit maximum for thi

In [17]:
merged_no_dup['PROC_DESC'] = merged_no_dup['PROCEDURE_DESCRIPTION'].map(unique_names)
print(merged_no_dup.shape)
merged_no_dup.drop(columns=['PROCEDURE_DESCRIPTION'],inplace=True)
print(merged_no_dup.shape)

(5571, 25)
(5571, 24)


/tmp/ipykernel_68207/394054059.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_68207/394054059.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
categorcal_cols.append('PROC_DESC')
categorcal_cols.remove('PROCEDURE_DESCRIPTION')

In [19]:
merged_no_dup.shape

(5571, 24)

In [20]:
def fix_cardinality(merged_no_dup, column, new_value ,in_freq = 30, ):
    value_counts = merged_no_dup[column].value_counts()

    freq_dict = dict(value_counts)
    for value, freq in freq_dict.items():
        if freq < in_freq:
            merged_no_dup.loc[merged_no_dup[column] == value, column] = new_value 
    return merged_no_dup

In [21]:
merged_no_dup = fix_cardinality(merged_no_dup,column='HCPCS_CODE', in_freq=50, new_value='G8979')
merged_no_dup = fix_cardinality(merged_no_dup,column='COUNTY', in_freq=50, new_value='COUNTY_X')
merged_no_dup = fix_cardinality(merged_no_dup,column='CODE', in_freq=173, new_value="CODE_X")
merged_no_dup.shape

(5571, 24)

In [22]:
merged_no_dup.to_csv('in_dataset.csv', index=False)

In [24]:
merged_no_dup.drop(columns=['REASON_CODE']).head().to_json('in_data_json.json', orient = 'split', compression = 'infer', index = 'true')

In [25]:
categorcal_cols = [col for col in categorcal_cols if col != 'REASON_CODE']

dumy_encoded = pd.get_dummies(merged_no_dup,columns=categorcal_cols,drop_first=True)
dumy_encoded.shape

(5571, 565)

In [26]:
norm_features = []
desc = dumy_encoded.describe()
for col in desc.columns:
    if desc.loc[:,col]['std'] > 1:
        norm_features.append(col)
        #print(col)

norm_features.remove('REASON_CODE')
norm_features = sorted(norm_features)
print(f' Features to be Normalized are : {norm_features}')

 Features to be Normalized are : ['Age', 'SERVICE_UNIT_QUANTITY', 'TOTAL_CHARGES']


In [27]:
scaler = MinMaxScaler()
scaler.fit(dumy_encoded[norm_features])

scaled = scaler.transform(dumy_encoded[norm_features])
dumy_encoded['Age'] =  scaled[:,0]
dumy_encoded['SERVICE_UNIT_QUANTITY'] = scaled[:,1]
dumy_encoded['TOTAL_CHARGES'] = scaled[:,2]
dumy_encoded.shape

(5571, 565)

In [ ]:
{
	"SERVICE_UNIT_QUANTITY":10,
	"Age":84,
	"TOTAL_CHARGES":1638.06,
	"BILL_TYPE_CODE":131,
	"HCPCS_CODE":"G8979",
	"GENDER":"female",
	"RACE":"white",
	"STATE":"Utah",
	"COUNTY":"COUNTY_X",
	"CODE":"CODE_X",
	"PRESENT_ON_ADMIT":0,
	"DUAL_STATUS":0,
	"ENCOUNTER_TYPE":"Other",
	"PAYERS":"Medicare",
	"PROCEDURE_CODE":"0W3P8ZZ",
	"PLACE_OF_SERVICE_CODE":23,
	"REVENUE_CENTER_CODE":636,
	"DISCHARGE_DISPOSITION_CODE":1,
	"MEDICARE_STATUS":10,
	"ADMIT_SOURCE_CODE":2,
	"ADMIT_TYPE_CODE":1,
	"MS_DRG":470,
	"PROC_DESC":"PD_1"
}

In [28]:
joblib.dump(scaler,'mnmx_scaler.joblib')

['mnmx_scaler.joblib']

2023-05-11 14:38:51,599 - clearml.Task - INFO - Completed model upload to http://3.111.229.37:8081/RCM_Claims_Denial_mgmt/EDA_Pandas_Profiling.e951dae3b9ad414a9b8ff59b8fd6c90f/models/mnmx_scaler.joblib


In [29]:
# add and upload local file artifac
task.upload_artifact(
    'local file', 
    artifact_object='mnmx_scaler.joblib'
)
task.close()

In [30]:
dumy_encoded.to_csv('preprocessed_cat_encoded.csv', index=False)

In [31]:
# Preprocessing code here
child_dataset = Dataset.create(
  dataset_name='claims_denial',
  dataset_project='RCM_Claims_Denial_mgmt', 
  parent_datasets=[dataset.id],
  dataset_version="1.0.1",
)
child_dataset.add_files('preprocessed_cat_encoded.csv')
child_dataset.upload()
child_dataset.finalize()

ClearML results page: http://3.111.229.37:8080/projects/0c1880d1adfb4bf3b970da71fa704cbc/experiments/5029b29e001945cf8514b2462fd1c1d2/output/log
ClearML dataset page: http://3.111.229.37:8080/datasets/simple/0c1880d1adfb4bf3b970da71fa704cbc/experiments/5029b29e001945cf8514b2462fd1c1d2
Uploading dataset changes (1 files compressed to 194.73 KiB) to http://3.111.229.37:8081
File compression and upload completed: total size 194.73 KiB, 1 chunk(s) stored (average size 194.73 KiB)


True

In [32]:
child_dataset.id

'5029b29e001945cf8514b2462fd1c1d2'